In [1]:
%run Common.ipynb

C:\Source\CSIRO\HealthyHabitatAI\.env
healthyhabitatanimals
healthyhabitatparragrass
ghNvkPl4DYLUUJh55y4pSg3BVV8TtPcHcX96P/zaDPXr9uuREdkGSCaao59d6xfZOfg4l9um1qfBXuNM9VGTTg==
jpEJGrFNi6PNLGOOD62rFBxjHGKTR4+DcppQNQQXi3P/GFmINKfqf8EZJoo73doIAnntv03ZSUsAC442IRR87Q==
processed
processed
f8fc2cd4db6b45a59d1ddd69a8685ddd
4610573e-864f-4f53-9f6a-c61ce520ed5a
5aeb092f-e91d-400f-bd88-e9516b108f20


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from PIL import Image, ImageFilter

In [3]:
%matplotlib inline

### Blob Storage

#### Para Grass Containers - *Site-Season*

In [4]:
para_grass_containers = blob_service_list_containers(healthy_habitat_paragrass_storage_name, healthy_habitat_paragrass_storage_key)

for container_name in para_grass_containers:
    print(container_name)

cannon-hill-bangkerreng
cannon-hill-kunumeleng
cannon-hill-wurrkeng
jabiru-dreaming-bangkerreng
jabiru-dreaming-kunumeleng
jabiru-dreaming-wurrkeng
resized
ubir-bangkerreng
ubir-kunumeleng
ubir-wurrkeng


### Resize

In [ ]:
for container_name in para_grass_containers:
    if container_name == 'ubir-kunumeleng':
        blobs = blob_service_list_blobs(healthy_habitat_paragrass_storage_name, healthy_habitat_paragrass_storage_key, container_name)
        
        folders = set([blob.split('/')[0] for blob in blobs])
        
        for folder in folders:
            interim_folder_path = os.path.join(Path(os.getcwd()).parent, 'data', 'interim', container_name, folder)
            print(interim_folder_path)
            if not os.path.exists(interim_folder_path):
                os.makedirs(interim_folder_path)
                
        for blob_name in blobs:
            folder = blob_name.split('/')[0]
            file = blob_name.split('/')[1]

            # https://docs.microsoft.com/en-us/rest/api/storageservices/understanding-the-table-service-data-model#characters-disallowed-in-key-fields
            row_key = '{0}-{1}'.format(folder, file)
            
            filter = "PartitionKey eq '{0}' and RowKey eq '{1}'".format(container_name, row_key)
            
            entities = table_service_query_entities(healthy_habitat_paragrass_storage_name,
                                                    healthy_habitat_paragrass_storage_key,
                                                    healthy_habitat_paragrass_processed_table_name,
                                                    filter)
            
            count = len([entity for entity in entities])

            if count == 0:
                print('Processing {0}'.format(blob_name))
                file_path = os.path.join(interim_folder_path, file)

                if not os.path.exists(file_path):
                    blob_service_get_blob_to_path(healthy_habitat_paragrass_storage_name,
                                                  healthy_habitat_paragrass_storage_key,
                                                  container_name,
                                                  blob_name,
                                                  file_path)

                edge = 1144
                
                image = Image.open(file_path)
                image = image.crop((0, 0, image.size[1], image.size[1])) #left, upper, right, lower
                image = image.resize((edge, edge), resample=Image.LANCZOS)
                image = image.filter(ImageFilter.BoxBlur(3))

                buffer = io.BytesIO()
                image.save(buffer, format='JPEG')

                path = '{0}/{1}/{2}'.format(container_name, folder, file)

                blob_service_create_blob_from_bytes(healthy_habitat_paragrass_storage_name,
                                                    healthy_habitat_paragrass_storage_key,
                                                    'resized',
                                                    path,
                                                    buffer.getvalue())

                entity = {'PartitionKey': container_name, 'RowKey': row_key, 'SemanticSegmentation': True}

                table_service_insert(healthy_habitat_paragrass_storage_name,
                                     healthy_habitat_paragrass_storage_key,
                                     healthy_habitat_paragrass_processed_table_name,
                                     entity)
                
                os.remove(file_path)

C:\Source\CSIRO\HealthyHabitatAI\data\interim\ubir-kunumeleng\2018-12-05-1200
Processing 2018-12-05-1200/DJI_0002.JPG
Processing 2018-12-05-1200/DJI_0003.JPG
Processing 2018-12-05-1200/DJI_0004.JPG


In [ ]:
tagged_image_count = trainer.get_tagged_image_count(custom_vision_para_grass_project_id)
print('Tagged image count:', tagged_image_count)

untagged_image_count = trainer.get_untagged_image_count(custom_vision_para_grass_project_id)
print('Untagged image count:', untagged_image_count)

print('Image count', tagged_image_count + untagged_image_count)